# Comparación

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from ydata_profiling import ProfileReport
import sklearn
import datetime

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_curve, auc

from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, classification_report
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_csv("data.csv")

In [5]:
data.loc[:, 'Popularity'] = data['Popularity'].astype(int)
X = data.drop('Popularity', axis=1)
y = data['Popularity']
X = pd.get_dummies(X)
makes = data['Make']
X_train, X_test, y_train, y_test, makes_train, makes_test = train_test_split(X, y, makes, test_size=0.2, random_state=42, stratify=y)

rf = RandomForestClassifier(random_state=42)

rf_param_distributions = {
    'n_estimators': [100, 200],
    'max_features': [None, 'sqrt', 'log2'],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

rf_random_search = RandomizedSearchCV(estimator=rf, param_distributions=rf_param_distributions,
                                      n_iter=10, cv=2, verbose=2, random_state=42, n_jobs=-1)

rf_random_search.fit(X_train, y_train)
best_rf = rf_random_search.best_estimator_

y_pred_rf = best_rf.predict(X_test)
print("Random Forest con mejores Hiperparametros:")
print(f"Exactitud: {accuracy_score(y_test, y_pred_rf)}")
print(classification_report(y_test, y_pred_rf))

results_rf = pd.DataFrame({'Make': makes_test, 'Predicted_Popularity': y_pred_rf})

avg_popularity_rf = results_rf.groupby('Make')['Predicted_Popularity'].mean().reset_index()

top_make_rf = avg_popularity_rf.loc[avg_popularity_rf['Predicted_Popularity'].idxmax()]

print("Mejor marca según Random Forest:")
print(top_make_rf)
avg_popularity_rf_sorted = avg_popularity_rf.sort_values(by='Predicted_Popularity', ascending=False)

top_3_makes_rf = avg_popularity_rf_sorted.head(3)
print("Tres marcas más populares según Random Forest:")
print(top_3_makes_rf)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Random Forest con mejores Hiperparametros:
Exactitud: 1.0
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1
          26       1.00      1.00      1.00         1
          61       1.00      1.00      1.00        31
          67       1.00      1.00      1.00         3
          86       1.00      1.00      1.00         6
         105       1.00      1.00      1.00        10
         113       1.00      1.00      1.00         1
         130       1.00      1.00      1.00         3
         155       1.00      1.00      1.00        25
         190       1.00      1.00      1.00        66
         204       1.00      1.00      1.00        50
         210       1.00      1.00      1.00        15
         238       1.00      1.00      1.00        12
         258       1.00      1.00      1.00        29
         259       1.00      1.00      1.00        19
         376    

In [6]:
data.loc[:, 'Popularity'] = data['Popularity'].astype(int)

X = data.drop('Popularity', axis=1)
y = data['Popularity']
X = pd.get_dummies(X)

makes = data['Make']

X_train, X_test, y_train, y_test, makes_train, makes_test = train_test_split(X, y, makes, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(random_state=42)

svm_param_distributions = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

svm_random_search = RandomizedSearchCV(estimator=svm, param_distributions=svm_param_distributions,
                                       n_iter=10, cv=2, verbose=2, random_state=42, n_jobs=-1)

svm_random_search.fit(X_train_scaled, y_train)
best_svm = svm_random_search.best_estimator_

y_pred_svm = best_svm.predict(X_test_scaled)
print("SVM con mejores Hiperparametros:")
print(f"Exactitud: {accuracy_score(y_test, y_pred_svm)}")
print(classification_report(y_test, y_pred_svm))

results_svm = pd.DataFrame({'Make': makes_test, 'Predicted_Popularity': y_pred_svm})

avg_popularity_svm = results_svm.groupby('Make')['Predicted_Popularity'].mean().reset_index()

top_make_svm = avg_popularity_svm.loc[avg_popularity_svm['Predicted_Popularity'].idxmax()]

print("Mejor marca según SVM:")
print(top_make_svm)
avg_popularity_svm_sorted = avg_popularity_svm.sort_values(by='Predicted_Popularity', ascending=False)

top_3_makes_svm = avg_popularity_svm_sorted.head(3)

print("Tres marcas más populares según SVM:")
print(top_3_makes_svm)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
SVM con mejores Hiperparametros:
Exactitud: 1.0
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1
          26       1.00      1.00      1.00         1
          61       1.00      1.00      1.00        31
          67       1.00      1.00      1.00         3
          86       1.00      1.00      1.00         6
         105       1.00      1.00      1.00        10
         113       1.00      1.00      1.00         1
         130       1.00      1.00      1.00         3
         155       1.00      1.00      1.00        25
         190       1.00      1.00      1.00        66
         204       1.00      1.00      1.00        50
         210       1.00      1.00      1.00        15
         238       1.00      1.00      1.00        12
         258       1.00      1.00      1.00        29
         259       1.00      1.00      1.00        19
         376       1.00   

Al revisar la curva de aprendizaje se concluye que ambos modelos estan aprendiendo bien de los datos de entrenamiento y generaliza bien a datos no vistos. Esto sugiere que el modelo no está sufriendo de overfitting significativo en los datos de entrenamiento y que está generalizando bien. Por esto no se deberia recolectar más datos, ajustar los hiperparámetros o probar otros algoritmos ya que ambos algoritmos de Random Forest y SVM no presenta problemas significativos de overfitting, underfitting o bias.

In [7]:
from scipy.stats import pearsonr

correlation_coef, p_value = pearsonr(y_pred_rf, y_pred_svm)


print("Coeficiente de correlación de Pearson entre las predicciones de Random Forest y SVM:", correlation_coef)


if p_value < 0.05:
    print("El valor p es significativamente bajo, lo que indica una correlación estadísticamente significativa entre las predicciones de los dos modelos.")
    if correlation_coef > 0:
        print("Hay una correlación positiva entre las predicciones de los modelos.")
    elif correlation_coef < 0:
        print("Hay una correlación negativa entre las predicciones de los modelos.")
    else:
        print("No hay correlación entre las predicciones de los modelos.")
else:
    print("El valor p no es lo suficientemente bajo para ser considerado estadísticamente significativo.")

Coeficiente de correlación de Pearson entre las predicciones de Random Forest y SVM: 0.9999999999999999
El valor p es significativamente bajo, lo que indica una correlación estadísticamente significativa entre las predicciones de los dos modelos.
Hay una correlación positiva entre las predicciones de los modelos.


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Para Random Forest
y_pred_rf = best_rf.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print("Métricas para Random Forest:")
print(f"Exactitud: {accuracy_rf}")
print(f"Precisión: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1-score: {f1_rf}")

#Para SVM
y_pred_svm = best_svm.predict(X_test_scaled)

accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')

print("\nMétricas para SVM:")
print(f"Exactitud: {accuracy_svm}")
print(f"Precisión: {precision_svm}")
print(f"Recall: {recall_svm}")
print(f"F1-score: {f1_svm}")

Métricas para Random Forest:
Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
F1-score: 1.0

Métricas para SVM:
Exactitud: 1.0
Precisión: 1.0
Recall: 1.0
F1-score: 1.0


In [10]:
from sklearn.model_selection import cross_val_score, KFold

def evaluate_model(model, X, y, cv):
    accuracy_scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(model, X, y, cv=cv, scoring='precision_weighted')
    recall_scores = cross_val_score(model, X, y, cv=cv, scoring='recall_weighted')
    f1_scores = cross_val_score(model, X, y, cv=cv, scoring='f1_weighted')
    
    return {
        'Accuracy': accuracy_scores.mean(),
        'Precision': precision_scores.mean(),
        'Recall': recall_scores.mean(),
        'F1-score': f1_scores.mean()
    }


k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# Evaluar Random Forest con k-fold cross-validation
rf_scores = evaluate_model(best_rf, X, y, k_fold)


svm_scores = evaluate_model(best_svm, X_train_scaled, y_train, k_fold)


print("Resultados de k-fold cross-validation para Random Forest:")
for metric, score in rf_scores.items():
    print(f"{metric}: {score}")

print("\nResultados de k-fold cross-validation para SVM:")
for metric, score in svm_scores.items():
    print(f"{metric}: {score}")

D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1509: Undefine

Resultados de k-fold cross-validation para Random Forest:
Accuracy: 0.9983920075681038
Precision: 0.9973828371659932
Recall: 0.9983920075681038
F1-score: 0.9978260391505674

Resultados de k-fold cross-validation para SVM:
Accuracy: 0.9990724038963817
Precision: 0.998934519205827
Recall: 0.9990724038963817
F1-score: 0.9989128310894712


#### Conclusión
Al ambos modelos de aprendizaje presentar métricas iguales a 1 y al usar un analisis de concordancia de coeficiente de correlación de Pearson, nos damos cuenta de que ambos modelos son igual de precisos. Se opta por hacer una validación cruzada k-fold. En esta si nos da como resultado que el modelo SVM, es mas preciso que el Randon Forest. Por tanto se llega a la conclusion que SVM es el modelo de aprendizaje de datos mas adecuado.